In [1]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke([HumanMessage(content="Hi, i'm bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6fc1df3-b5ea-4887-8a58-42505770a7bf-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState:
    response = model.invoke(state['messages'])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {
    "configurable": {
        "thread_id": 1
    }
}

In [13]:
query = "Hi, i'm bob"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [14]:
output["messages"]

[HumanMessage(content="Hi, i'm bob", additional_kwargs={}, response_metadata={}, id='8ecb0a98-7fab-40a9-b978-d6b7261d3837'),
 AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-25a34659-3c18-440a-83cd-ddc79fead7d2-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

In [15]:
query = "who am i?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You're Bob! If there's something more specific you'd like to explore about yourself or if you have any questions, feel free to share!


# Postgres

In [14]:
import uuid
import os

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_postgres import PostgresChatMessageHistory
import psycopg
import psycopg_pool


# Establish a synchronous connection to the database
# (or use psycopg.AsyncConnection for async)
conn_info = f"host={os.getenv('DATABASE_HOST')} port={os.getenv('DATABASE_PORT')} dbname={os.getenv('DATABASE_NAME')} user={os.getenv('DATABASE_USER')} password={os.getenv('DATABASE_PASSWORD')}"

print(conn_info)

with psycopg_pool.ConnectionPool(conn_info) as pool:
    with pool.connection() as conn:
        # Create the table schema (only needs to be done once)
        table_name = "chat_history"
        PostgresChatMessageHistory.create_tables(conn, table_name)

        # session_id = str(uuid.uuid4())
        session_id = "65a490d8-3b1b-4d1c-81ca-38b11d00ace7"

        # Initialize the chat history manager
        chat_history = PostgresChatMessageHistory(
            table_name,
            session_id,
            sync_connection=conn
        )

        # Add messages to the chat history
        chat_history.add_messages([
            SystemMessage(content="Meow"),
            AIMessage(content="woof"),
            HumanMessage(content="bark"),
        ])

        print(chat_history.get_messages())

host=localhost port=5433 dbname=ai-buddy user=root password=root
[SystemMessage(content='Meow', additional_kwargs={}, response_metadata={}), AIMessage(content='woof', additional_kwargs={}, response_metadata={}), HumanMessage(content='bark', additional_kwargs={}, response_metadata={}), SystemMessage(content='Meow', additional_kwargs={}, response_metadata={}), AIMessage(content='woof', additional_kwargs={}, response_metadata={}), HumanMessage(content='bark', additional_kwargs={}, response_metadata={}), SystemMessage(content='Meow', additional_kwargs={}, response_metadata={}), AIMessage(content='woof', additional_kwargs={}, response_metadata={}), HumanMessage(content='bark', additional_kwargs={}, response_metadata={})]
